## LLM Integration

Once all the steps were developed:

- Embedding Service
- Ingestion Pipeline
- Context retrieval

Now its time to create the last part of the RAG-LLM technique. Send the context and the user's query to the LLM in order to the LLM to generate an answer.

This time, I'll be using ChatGPT LLM's, but also can work with Google LLM and others.


In [16]:
from google import genai
from google.genai import types

import sys

sys.path.append("..")

from rag_llm_energy_expert.credentials import get_qdrant_config, get_llm_config
from rag_llm_energy_expert.search.searchers import semantic_search
from rag_llm_energy_expert.llm.chat import create_chat_session, generate_response

In [17]:
qdrant_config=get_qdrant_config()
llm_config=get_llm_config()
collection_name = qdrant_config.COLLECTION_NAME + qdrant_config.COLLECTION_VERSION

## Connecting the GENAI client

code from: https://ai.google.dev/gemini-api/docs/text-generation

In [18]:
llm_client = genai.Client(api_key=llm_config.API_KEY.get_secret_value())

Generating multi-turn conversations

The chat format enables users to step incrementally toward answers and to get help with multipart problems.

In [19]:
# Creates a new chat session
chat = llm_client.chats.create(model=llm_config.MODEL)

In [20]:
response = chat.send_message("Hi, im 25 years old")
print(response.text)

Okay! That's great. How can I help you today? Are you looking for information about something specific, or just looking to chat? Tell me what's on your mind.



In [21]:
response = chat.send_message("If I am 5 years older than my sister. How old is she?")
response.text

'Since you are 25 years old and 5 years older than your sister, she is 25 - 5 = 20 years old.\n\nSo, your sister is **20 years old**.\n'

In [22]:
for message in chat.get_history():
    print(f'role - {message.role}',end=": ")
    print(message.parts[0].text)

role - user: Hi, im 25 years old
role - model: Okay! That's great. How can I help you today? Are you looking for information about something specific, or just looking to chat? Tell me what's on your mind.

role - user: If I am 5 years older than my sister. How old is she?
role - model: Since you are 25 years old and 5 years older than your sister, she is 25 - 5 = 20 years old.

So, your sister is **20 years old**.



### Configuring parameters

Every prompt sent to the model includes parameters that control how the model generates responses. You can configure these parameters, por let the model use the default options

In [23]:
# Creates a new chat session
chat2 = llm_client.chats.create(
    model=llm_config.MODEL,
    config=types.GenerateContentConfig(
        max_output_tokens=500,
        temperature=0.1
    )
    )

In [24]:
responses = chat2.send_message(
    message = "Hi, If I have 5 apples, and 2 pears, and for all of them I paid 10 USD, if the apples costs 1 USD, how much are the pears?"
)
print(responses.text)

Here's how to solve the problem:

*   **Cost of apples:** 5 apples * $1/apple = $5
*   **Cost of pears:** $10 (total) - $5 (apples) = $5
*   **Cost per pear:** $5 / 2 pears = $2.50/pear

**Answer:** The pears cost $2.50 each.


More model parameters can be found [here](https://ai.google.dev/gemini-api/docs/text-generation)

### System Instructions

System instructions let you steer the behaviour of a model baesd on you specific use case. When you provide system instructions, you give the model additional context to help it understand the task and generate more customized responses. The model should adhere to the system instructions over the full iteraction with the user, enabling you to specify product-level behaviour separete from the prompts provided by end users.

In [25]:
# Creates a new chat session
chat3 = llm_client.chats.create(
    model=llm_config.MODEL,
    config=types.GenerateContentConfig(
        max_output_tokens=500,
        temperature=0.1,
        system_instruction="You are a Mexican energy expert that solves doubts of clients. You must be as direct as possible. Your responses" \
        "shall not be longer than 2 paragraphs (5 lines each)." \
        "The responses shall be based on the context provided. If you don't know the answer, tell that you don't know." \
        "Answer the user's questions in the same language as they're asked."
    )
    )

In [26]:
query = "En el nuevo modelo, cómo se considera a Pemex?"

Semantic Search of the available info in the vector DB

In [27]:

print(semantic_search(
    query=query,
    embedding_model_name=None,
    chunk_overlap=0,
    documents_limit=5,
    collection_name=collection_name
))

2025-04-22 10:42:53.290 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query:37 - Preprocessing query...
2025-04-22 10:42:53.292 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query:74 - Generating embeddings...
2025-04-22 10:42:55.815 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query:87 - Embeddings generated successfully
2025-04-22 10:42:55.815 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query:95 - Preparing embeddings for vector search
2025-04-22 10:42:55.815 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query:106 - Query preprocessed successfully
2025-04-22 10:42:56.223 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query_results:120 - Processing query results...
2025-04-22 10:42:56.223 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query_results:131 - Query results processed


- Pemex y la CFE se mantienen como empresas 100% mexicanas y 100% del Estado, cuya organización, administración, organización y estructura corporativa serán acordes con las mejores prácticas internacionales.  
**Legislación Secundaria**  
- Pemex y CFE contarán con un régimen presupuestario especial y exclusivo que les otorga autonomía presupuestaria y las sujeta sólo al balance financiero y al techo de servicios personales.  
- Para el manejo de su deuda, Pemex y CFE se regirán por lo dispuesto en un artículo especial de la Ley General de Deuda Pública. Ambas empresas podrán realizar negociaciones, así
como contratar los financiamientos externos e internos que requieran, para lo que deberán
coordinarse con la SHCP, sin requerir de su autorización.  
- El Gobierno de la República dejará de ser el “administrador principal” de los órganos de gobierno de las Empresas Productivas del Estado para asumir el rol de propietario.

- Toda la información geológica del país será entregada a CNH, q

In [28]:
question = "PEMEX podrá migrar a la nueva modalidad de contratación"

response = chat3.send_message(message=question,
                              config = types.GenerateContentConfig(
                                  temperature=0.5,
                                  system_instruction="You are a Mexican energy expert that solves doubts of clients. Your responses" \
        "shall not be longer than 2 paragraphs (5 lines each)." \
        "The responses shall be based on the context provided. If you don't know the answer, tell that you don't know." \
        "Answer the user's questions in the same language as they're asked. Try to generate friendly answers"\
        f"""Context: {semantic_search(query=question,
                                    embedding_model_name=None,
                                    chunk_overlap=0,
                                    collection_name = collection_name,
                                    documents_limit = 5
                                    )}"""
                              ))
print(response.text)

2025-04-22 10:42:56.254 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query:37 - Preprocessing query...
2025-04-22 10:42:56.256 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query:74 - Generating embeddings...
2025-04-22 10:42:57.805 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query:87 - Embeddings generated successfully
2025-04-22 10:42:57.806 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query:95 - Preparing embeddings for vector search
2025-04-22 10:42:57.809 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query:106 - Query preprocessed successfully
2025-04-22 10:42:57.931 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query_results:120 - Processing query results...
2025-04-22 10:42:57.933 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query_results:131 - Query results processed


¡Hola! Sí, Pemex tiene la posibilidad de migrar a la nueva modalidad de contratación. Esto le permitirá poner en producción yacimientos de hidrocarburos que actualmente no se explotan por falta de inversión, capacidad de ejecución y tecnología.


## Using the functions generated

In [29]:
chat = create_chat_session()

2025-04-22 10:42:58.933 | INFO     | rag_llm_energy_expert.llm.chat:create_chat_session:42 - Creating a new chat session...
2025-04-22 10:42:58.936 | INFO     | rag_llm_energy_expert.llm.chat:create_chat_session:51 - Chat session created successfully.


In [30]:
query="PEMEX podrá migrar a la nueva modalidad de contratación"

print(generate_response(prompt=query, chat_session=chat, temperature=0.1))

2025-04-22 10:42:58.953 | INFO     | rag_llm_energy_expert.llm.chat:generate_response:74 - Generating response...
2025-04-22 10:42:58.955 | INFO     | rag_llm_energy_expert.llm.chat:generate_response:75 - Retrieving context...
2025-04-22 10:42:58.957 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query:37 - Preprocessing query...
2025-04-22 10:42:58.957 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query:74 - Generating embeddings...
2025-04-22 10:43:00.089 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query:87 - Embeddings generated successfully
2025-04-22 10:43:00.098 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query:95 - Preparing embeddings for vector search
2025-04-22 10:43:00.102 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_query:106 - Query preprocessed successfully
2025-04-22 10:43:00.221 | INFO     | rag_llm_energy_expert.search.searchers_auxiliars:process_

Sí, se establece la posibilidad de que la Nación otorgue asignaciones o contratos a Pemex, lo que le permitiría poner en producción yacimientos de hidrocarburos que actualmente están ociosos por falta de inversión, capacidad de ejecución y tecnología.
